In [ ]:
import requests
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Government bond prices

In [ ]:
dates = pd.date_range(start='2017-01-01', end='2020-12-01')

In [ ]:
def get_prices(date, i):
    base = 'https://iss.moex.com/iss/history/engines/stock/markets/bonds/boards/TQOB/securities.json?date='
    url = f'{base}{date}'
    try:
        resp = requests.get(url).json()
        df = pd.DataFrame(resp['history']['data'], columns=resp['history']['metadata'])
        df['DATE'] = [date for i in range(len(df))]
        return df
    except:
        print('oops', i)

In [ ]:
df_main = pd.DataFrame()

for i, date in tqdm(enumerate(dates)):
    df_main = df_main.append(get_prices(date, i))

# Specifications of government bonds

In [ ]:
def get_instr(id_):
    url = f'https://iss.moex.com/iss/securities/{id_}.json' 
    response = requests.get(url).json()
    info = []
    info.append(id_)
    columns = np.array(response['description']['data'])[:,0]
    row = np.array(response['description']['data'])[:,2]
    mat_ind = np.where(columns == 'MATDATE')
    mat_data = row[mat_ind][0]
    volume_ind = np.where(columns == 'ISSUESIZE')
    volume_data = row[volume_ind][0]
    coupon_ind = np.where(columns == 'COUPONFREQUENCY')
    coupon_data = row[coupon_ind][0]
    info.append(mat_data)
    info.append(volume_data)
    info.append(coupon_data)
    return info

In [ ]:
sec_data = []

for sec_id in tqdm(data['SECID'].unique()):
    sec_data.append(get_instr(sec_id))

# VIX 

In [ ]:
def get_vix(index_name, start):
    url = f'https://iss.moex.com/iss/history/engines/stock/markets/index/securities/{index_name}.json?from=2015-01-01&start={start}'
    resp = requests.get(url).json()
    df = pd.DataFrame(resp['history']['data'], columns=resp['history']['columns'])
    df['DELTA'] = (df['CLOSE'] - df['OPEN'])/df['OPEN']
    return df[['TRADEDATE', 'CLOSE', 'DELTA']]

In [ ]:
df_vix = pd.DataFrame(columns = ['TRADEDATE', 'CLOSE', 'DELTA'])

for start in tqdm(np.arange(0, 5000, 100)):
    df_vix = df_vix.append(get_vix('RVI', start), ignore_index=True)

# Exchange rates

In [ ]:
def get_dollas(start):
    url = f'https://iss.moex.com/iss/history/engines/stock/markets/index/securities/{index_name}.json?from=2015-01-01&start={start}'
    resp = requests.get(url).json()
    df = pd.DataFrame(resp['securities']['data'], columns = resp['securities']['columns'])
    return df[['tradedate', 'rate']]

# Corporate bond prices

In [ ]:
def get_prices(date):
    base = 'https://iss.moex.com/iss/history/engines/stock/markets/bonds/securities.json?date='
    start = 0
    url = f'{base}{date}&start={start}'
    resp = requests.get(url).json()
    df = pd.DataFrame()
    while resp['history']['data']:
        df = df.append(pd.DataFrame(resp['history']['data'], columns=resp['history']['columns']))
        start += 100
        url = f'{base}{date}&start={start}'
        resp = requests.get(url).json()
    return df

In [ ]:
df_main = pd.DataFrame()
for date in tqdm(pd.date_range(start='2015-01-01', end='2017-01-01', freq='1D')):
    df_main = df_main.append(get_prices(date))

# Bond specifications

In [ ]:
big_df = pd.read_csv('CORP_prices.csv')

In [ ]:
def get_info(id_):
    url = f'https://iss.moex.com/iss/securities/{id_}.json'
    resp = requests.get(url).json()
    df = pd.DataFrame(resp['description']['data'], columns=resp['description']['columns']).set_index('name')
    
    try:
        qual = df.loc['ISQUALIFIEDINVESTORS']['value']
    except:
        qual = np.nan
    
    try:
        list_level = df.loc['LISTLEVEL']['value']
    except:
        list_level = np.nan
    
    try:
        type_name = df.loc['TYPE']['value']
    except:
        type_name = np.nan
    
    try:
        issue_size = df.loc['ISSUESIZE']['value']
    except:
        issue_size = np.nan
        
    try:
        coupon_freq = df.loc['COUPONFREQUENCY']['value']
    except:
        coupon_freq = np.nan
        
    return [id_, qual, list_level, type_name, issue_size, coupon_freq]

In [ ]:
def get_matdate(id_):
    url = f'https://iss.moex.com/iss/securities/{id_}.json'
    resp = requests.get(url).json()
    df = pd.DataFrame(resp['description']['data'], columns=resp['description']['columns']).set_index('name')
   
    try:
        matdate = df.loc['MATDATE']['value']
    except:
        matdate = np.nan
        
    return [id_, matdate]

In [ ]:
info = []
for id_ in tqdm(df_main['SECID'].unique()):
    info.append(get_info(id_))

In [ ]:
info = []
for id_ in tqdm(df_main['SECID'].unique()):
    info.append(get_matdate(id_))

# Corporate bond indices

In [ ]:
def get_index(index_name, start):
    url = f'https://iss.moex.com/iss/history/engines/stock/markets/index/securities/{index_name}.json?from={start}'
    resp = requests.get(url).json()
    df = pd.DataFrame(resp['history']['data'], columns=resp['history']['columns'])
    return df[['TRADEDATE', 'CLOSE']]

In [ ]:
df_1y = pd.DataFrame(columns = ['TRADEDATE', 'CLOSE'])
df_3y = pd.DataFrame(columns = ['TRADEDATE', 'CLOSE'])
df_5y = pd.DataFrame(columns = ['TRADEDATE', 'CLOSE'])
df_33y = pd.DataFrame(columns = ['TRADEDATE', 'CLOSE'])

In [ ]:
dates = pd.date_range(start='2015-01-01', end='2021-01-01', freq='50D')

In [ ]:
for start in dates:
    df_1y = df_1y.append(get_index('RUCBITR1Y', str(start).split()[0]), ignore_index=True)
df_1y.columns = ['TRADEDATE', 'RUCBITR1Y']

In [ ]:
for start in tqdm(dates):
    df_3y = df_3y.append(get_index('RUCBTR3Y', str(start).split()[0]), ignore_index=True)
df_3y.columns = ['TRADEDATE', 'RUCBTR3Y']

In [ ]:
for start in tqdm(dates):
    df_5y = df_5y.append(get_index('RUCBTR5Y', str(start).split()[0]), ignore_index=True)
df_5y.columns = ['TRADEDATE', 'RUCBTR5Y']

In [ ]:
for start in tqdm(dates):
    df_33y = df_33y.append(get_index('RUCBITR3+', str(start).split()[0]), ignore_index=True)
df_33y.columns = ['TRADEDATE', 'RUCBITR3+']

# IMOEX

In [ ]:
def get_index(index_name, start):
    url = f'https://iss.moex.com/iss/history/engines/stock/markets/index/securities/{index_name}.json?from=2017-01-01&start={start}'
    resp = requests.get(url).json()
    df = pd.DataFrame(resp['history']['data'], columns=resp['history']['columns'])
    return df[['TRADEDATE', 'CLOSE']]

In [ ]:
df_imoex = pd.DataFrame()
for start in tqdm(np.arange(0, 1200, 100)):
    df_imoex = df_imoex.append(get_index('IMOEX', start))
df_imoex.columns = ['TRADEDATE', 'IMOEX']

## Data on floating coupons, buy-back options, secured bonds from RusBonds

In [ ]:
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm.notebook import tqdm

In [ ]:
PATH = 'C:\Program Files (x86)\chromedriver.exe'

In [ ]:
driver = webdriver.Chrome(PATH)

In [ ]:
def get_info(isin):
    path = f'Corp_data_new//{isin}.csv'
    url = f'https://www.rusbonds.ru/srch_simple.asp?go=1&nick={isin}&emit=0&sec=0&status=&cat=0&per=0&rate=0&ctype=0&pvt=0&grnt=0&conv=0&amm=0&bpog=&epog=&brazm=&erazm=&bvip=&evip=&brep=&erep=&bemis=&eemis=&bstav=&estav=&bcvol=&ecvol=#rslt'

    driver.get(url)
    
    try:
        table = driver.find_element(By.CLASS_NAME, 'tbl_data.tbl_headgrid')
    except:
        pd.DataFrame([[isin, "Not Found", np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]]).to_csv(path, index=False)
        return
    
    table.find_element(By.TAG_NAME, 'tbody').find_element(By.TAG_NAME, 'a').click()
    
    soup = BeautifulSoup(driver.page_source)
    data = [i.text for i in soup.find(class_='tbl_data').find_all('tr')]
    try:
        issuer = soup.find('b', string='Эмитент:').parent.find('a').text
    except:
        issuer = np.nan
    
    secured = False
    amort = False
    early_1 = False
    early_2 = False
    coupon = False
    period = False
    extra = False
    
    for line in data:
        if 'Обеспечение' in line:
            secured = line
        elif 'ПОГАШЕНИЕ' in line and 'ДОСРОЧН.' not in line:
            amort = line
        elif 'ДОСРОЧН.' in line:
            early_1 = line
        elif 'Возможность досрочного погашения' in line:
            early_2 = line
        elif 'КУПОН' in line:
            coupon = line
        elif 'Периодичность выплат в год' in line:
            period = line
        elif 'Примечание'in line:
            extra = line
            
    if secured:
        secured = ''.join(secured.split(':')[1].split(u'\xa0')[:-1])
    else:
        secured = np.nan
        
    if amort:
        amort = amort.split('-')[-1]
    else:
        amort = np.nan
    
    if early_1:
        early_1 = 1
    else:
        early_1 = np.nan
    
    if coupon:
        coupon = coupon.split('-')[-1]
    else:
        coupon = np.nan
        
    if period:
        period = int(period.split(':')[-1])
    else:
        period = np.nan
    
    if not early_2:
        early_2 = np.nan
    
    if not extra:
        extra = np.nan
        
    sleep_time = np.random.randint(3, 8)
    #time.sleep(sleep_time)
        
    pd.DataFrame([[isin, issuer, secured, amort, early_1, early_2, coupon, period, extra]]).to_csv(path, index=False)

In [ ]:
for isin in tqdm(df_main['SECID'].unique()):
    get_info(isin)

# Data on defaults from RusBonds

In [ ]:
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm.notebook import tqdm

In [ ]:
PATH = 'C:\Program Files (x86)\chromedriver.exe'

In [ ]:
driver = webdriver.Chrome(PATH)

In [ ]:
def find_default(isin):
    url = f'https://www.rusbonds.ru/srch_simple.asp?go=1&nick={isin}&emit=0&sec=0&status=&cat=0&per=0&rate=0&ctype=0&pvt=0&grnt=0&conv=0&amm=0&bpog=&epog=&brazm=&erazm=&bvip=&evip=&brep=&erep=&bemis=&eemis=&bstav=&estav=&bcvol=&ecvol=#rslt'
    driver.get(url)
    
    try:
        table = driver.find_element(By.CLASS_NAME, 'tbl_data.tbl_headgrid')
    except:
        return pd.DataFrame([[isin, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN]], 
                            columns=['SECID', 'Дата дефолта', 'Дата исполненияобязательств',
       'Номер обязательства', 'Примечание', 'Тип дефолта',
       'Тип неисполненныхобязательств'])
    
    table.find_element(By.TAG_NAME, 'tbody').find_element(By.TAG_NAME, 'a').click()
    soup = BeautifulSoup(driver.page_source)
    if soup.find('table', class_='tbl_data tbl_headgrid'):
        
        table_data = [i.text for i in soup.find('table', class_='tbl_data tbl_headgrid').find_all('td')[:-1]]
        return pd.concat([pd.DataFrame([table_data[i:i+6] for i in range(6, len(table_data)-5, 6)], columns=table_data[:6]),
               pd.DataFrame([isin for i in range(int((len(table_data)-6)/6))], columns=['SECID'])], axis=1)
    else:
        
        return pd.DataFrame([[isin, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 'No Default']],
                           columns=['SECID', 'Дата дефолта', 'Дата исполненияобязательств',
       'Номер обязательства', 'Примечание', 'Тип дефолта',
       'Тип неисполненныхобязательств'])

In [ ]:
default_data = pd.DataFrame()

for isin in tqdm(df_main['SECID'].unique()):
    default_data = default_data.append(find_default(isin), sort=True)